<a href="https://colab.research.google.com/github/walkerjian/dev/blob/main/chatnessEvergreen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install httpx==0.27.2 --force-reinstall

In [ ]:
from google.colab import userdata
mykey=userdata.get('OPENAI_API_KEY')
print(mykey)

In [ ]:
import os
import json
from openai import OpenAI
from google.colab import auth

# Authenticate user
auth.authenticate_user()

client = OpenAI(
    api_key=mykey  # Replace with your API key
)
def load_conversation(file_path):
    """Load conversation history from a file."""
    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            try:
                print("Loading conversation from file...")
                print(file_path)
                return json.load(file)
            except json.JSONDecodeError:
                print("Conversation file is corrupted. Starting fresh.")
    return [{"role": "system", "content": "You are a helpful assistant."}]

def save_conversation(file_path, messages):
    """Save conversation history to a file."""
    with open(file_path, "w") as file:
        json.dump(messages, file, indent=4)

def continue_chat(messages, user_input, model="gpt-4"):
    """Continue the chat by sending the conversation context and the new user input."""
    messages.append({"role": "user", "content": user_input})
    try:
        # Create a chat completion
        chat_completion = client.chat.completions.create(
            messages=messages,
            model=model,
        )
        # Extract the assistant's response
        assistant_reply = chat_completion.choices[0].message.content
        print(f"\nAssistant: {assistant_reply}\n")
        # Append the assistant's reply to the conversation history
        messages.append({"role": "assistant", "content": assistant_reply})
        return messages
    except Exception as e:
        print(f"An error occurred: {e}")
        return messages


In [ ]:
# File for conversation history
filename = input("Enter the name of an existing conversation file to continue (or press Enter to start new): ").strip()

if filename and os.path.exists(filename):
    print(f"Loading conversation from {filename}...")
    conversation = load_conversation(filename)
else:
    if filename:
        print(f"File '{filename}' not found. Starting a new conversation.")
    conversation = [{"role": "system", "content": "You are a helpful assistant."}]
    filename = input("Enter a filename to save this new conversation: ").strip()


Just copy and paste this cell to continue the chat!

In [ ]:
# Get user input
user_message = input("You: ")

# Continue the conversation
conversation = continue_chat(conversation, user_message)